In [12]:
import spacy
import numpy as np
from collections import OrderedDict
nlp = spacy.load('en_core_web_lg')

# ms = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings['engine']]]), n=10)
# [nlp.vocab.strings[w] for w in ms[0][0]]

KeyboardInterrupt: 

In [ ]:
def get_most_similar(word, n=10):
    result = []
    result_num = 0
    estimation_factor = 2.0
    estimation = int(n * estimation_factor)
    while result_num < n:
        estimation = int((n - result_num) * estimation_factor) + estimation
        ms = nlp.vocab.vectors.most_similar(np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word]]]), n=estimation)
        stemmed_words_list = [nlp(nlp.vocab.strings[w].lower())[0].lemma_ for w in ms[0][0]]
        stemmed_words_set = list(OrderedDict.fromkeys(stemmed_words_list))
        result_num = len(stemmed_words_set)
        print(f"{result_num}/{estimation} retrieved")
    return stemmed_words_set[0:n]

In [ ]:
get_most_similar("engine", n=5)

6/20 retrieved


['engine', 'diesel', 'horsepower', 'motor', 'powerplant']

In [ ]:
from sense2vec import Sense2VecComponent

nlp = spacy.load("en_core_web_lg")
s2v = nlp.add_pipe("sense2vec")
s2v.from_disk("../../s2v_reddit_2019_lg")

In [ ]:
from convokit import Corpus, download
import convokit

corpus = Corpus(download('subreddit-askcarsales'))
corpus.print_summary_stats()

Dataset already exists at /Users/enoch/.convokit/downloads/subreddit-askcarsales
Number of Speakers: 41793
Number of Utterances: 651481
Number of Conversations: 47109


In [ ]:
utt_ids = list(corpus.utterances.keys())
utt_ids[0:20]

['1difca',
 '1di91i',
 '1di8e1',
 '1di6f6',
 '1dhumo',
 '1dhu9v',
 '1dhqpm',
 '1dgz2b',
 '1dgufc',
 '1dkiug',
 '1dkfa3',
 '1dkcbi',
 '1dkb0j',
 '1dk7mf',
 '1djz98',
 '1djt0z',
 '1dj3x5',
 '1diqzz',
 '1dmvty',
 '1dmvmy']

In [ ]:
global keywords
keywords_dict = {
    "power": ["engine~NOUN", "horsepower~NOUN", "emission~NOUN", "diesel~NOUN", "speedup~NOUN", "4WD~NOUN", "4-wheel-drive~NOUN", "four-wheel-drive~NOUN" ,"2WD~NOUN", "2-wheel-drive~NOUN", "two-wheel-drive~NOUN", "front-wheel-drive~NOUN"],
    "configuration": ["conditioner~NOUN",  "bluetooth~NOUN", "carplay~NOUN", "stereo~NOUN", "radio~NOUN", "seat~NOUN", "steering wheel~NOUN", "navigation~NOUN", "brake~NOUN", "monitor~NOUN", "assistant~NOUN"],
    "exterior": ["blue~ADJ", "red~ADJ", "black~ADJ", "white~ADJ", "silver~ADJ", "charcoal~ADJ", "metallic~ADJ", "pink~ADJ", "SUV~NOUN", "sedan~NOUN"],
    "price": ["budget~NOUN", "poor~ADJ", "cheap~ADJ", "luxury~ADJ", "price~NOUN", "moderate~ADJ", "expensive~ADJ"]
}
keywords = []
for k in keywords_dict.values():
    keywords += k
def car_filter(utt):
    filtered_vals = [k in utt.text for k in keywords]
    return any(filtered_vals)

In [ ]:
from convokit.text_processing import TextParser
qparser = TextParser(output_field='qparsed', input_field='clean_text', input_filter=car_filter, verbosity=50)
corpus = qparser.transform(corpus)

050/651481 utterances processed
100/651481 utterances processed
150/651481 utterances processed
200/651481 utterances processed
250/651481 utterances processed
300/651481 utterances processed
350/651481 utterances processed
400/651481 utterances processed
450/651481 utterances processed
500/651481 utterances processed
550/651481 utterances processed
600/651481 utterances processed
650/651481 utterances processed
700/651481 utterances processed
750/651481 utterances processed
800/651481 utterances processed
850/651481 utterances processed
900/651481 utterances processed
950/651481 utterances processed
1000/651481 utterances processed
1050/651481 utterances processed
1100/651481 utterances processed
1150/651481 utterances processed
1200/651481 utterances processed
1250/651481 utterances processed
1300/651481 utterances processed
1350/651481 utterances processed
1400/651481 utterances processed
1450/651481 utterances processed
1500/651481 utterances processed
1550/651481 utterances proces

In [ ]:
utt_ids = list(corpus.utterances.keys())
utt_ids[0:10]

['1difca',
 '1di91i',
 '1di8e1',
 '1di6f6',
 '1dhumo',
 '1dhu9v',
 '1dhqpm',
 '1dgz2b',
 '1dgufc',
 '1dkiug']

In [ ]:
import re
import json

patterns = {
        k: re.compile(
        "|".join(
            [f"([\S]*\s{x.replace('~NOUN', '')})" if '~NOUN' in x else f"({x.replace('~ADJ', '')}\s[\S]*)" for x in keywords_dict[k]]
        )
    ) for k in keywords_dict.keys()
}

with open("stop_words_english.json", "r") as f:
    stopwords = json.load(f)
    
stopwords_pattern = re.compile(
    "|".join(
        [f"(^{s.lower()}\s[\S]*)" for s in stopwords]
        )
)

In [16]:
from tqdm import trange, tqdm
filtered_corpus = {k: [] for k in patterns.keys()}
for i in trange(100000):
    text = corpus.get_utterance(utt_ids[i]).text
    if (text != ""):
        for feature_class, feature_pattern in patterns.items():
            matches = feature_pattern.findall(text)
            if len(matches) != 0:
                filtered_corpus[feature_class] += [x for x in matches[0] if x!=""]
        # print(pattern_match(text))

100%|██████████| 3057/3057 [00:12<00:00, 254.22it/s]


['2wd diesel',
 'fucking engine',
 "carb'ed engine",
 'Explorer 4WD',
 'Limited 4WD',
 'Dodge engine',
 'crappy engine',
 'LX 2WD',
 "'00 4WD",
 "Kizashi's engine"]

In [20]:
filtered_corpus_stop_removed = {k: [] for k in patterns.keys()}
for feature_class in filtered_corpus.keys():
    filtered_corpus[feature_class] = list(set(filtered_corpus[feature_class]))
for feature_class in filtered_corpus_stop_removed.keys():
    print(feature_class)
    for text in tqdm(filtered_corpus[feature_class]):
        matches = stopwords_pattern.findall(text.lower())
        if len(matches)!=0 and len([x for x in matches[0] if x!=""])!=0:
            continue
        if any([k in text for k in [":", ",", ".", "(", ")", ";", "?", "!", "/"]]):
            continue
        if text.split(" ")[0]=="-":
            continue
        if len(text) < 5:
            continue
        filtered_corpus_stop_removed[feature_class].append(text)
filtered_corpus_stop_removed["power"][0:10]

power


100%|██████████| 1293/1293 [00:05<00:00, 255.30it/s]


configuration


100%|██████████| 1059/1059 [00:04<00:00, 239.63it/s]


exterior


100%|██████████| 3130/3130 [00:11<00:00, 266.02it/s]


price


100%|██████████| 3057/3057 [00:12<00:00, 246.96it/s]


['2wd diesel',
 'fucking engine',
 "carb'ed engine",
 'Explorer 4WD',
 'Limited 4WD',
 'Dodge engine',
 'crappy engine',
 'LX 2WD',
 "'00 4WD",
 "Kizashi's engine"]

In [19]:
for k in filtered_corpus_stop_removed.keys():
    output = "\n".join(filtered_corpus_stop_removed[k])
    with open(f"data/{k}.txt", "w") as f:
        f.writelines(output)